In [58]:
import os
import cv2
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm
from torchsummary import summary

# data augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

# pretrained models
import torchvision
from torchvision import models, transforms

from functions import *

## Load in metadata

In [2]:
all_metadata = pd.read_csv('data/all_data_info.csv')

In [3]:
# select artists for the task
artists = ['John Singer Sargent', 'Pablo Picasso', 'Pierre-Auguste Renoir', 'Paul Cezanne', 'Camille Pissarro', 'Paul Gauguin', 
           'Claude Monet', 'Edgar Degas', 'Henri Matisse', 'Vincent van Gogh', 'Childe Hassam', 'Pyotr Konchalovsky', 'Martiros Saryan', 
           'Boris Kustodiev', 'Nicholas Roerich', 'Salvador Dali', 'Alfred Sisley', 'Henri Martin', 'Rene Magritte', 'Konstantin Korovin', 
           'Mary Cassatt', 'Gustave Loiseau', 'John Henry Twachtman', 'Georges Braque', 'Pierre Bonnard', "Georgia O'Keeffe", 
           'Gustave Caillebotte', 'Ilya Mashkov', 'Andy Warhol', 'Theo van Rysselberghe', 'Georges Seurat', 'Edward Hopper', 'Maxime Maufra', 
           'Diego Rivera', 'Henri-Edmond Cross', 'Robert Julian Onderdonk', 'Guy Rose', 'Andre Derain', 'Willard Metcalf', 'Frida Kahlo', 
           'Paul Signac', 'William James Glackens', 'Frantisek Kupka', 'Julian Alden Weir', 'Paul Serusier', 'Max Pechstein', 
           'Victor Borisov-Musatov', 'Armand Guillaumin', 'Spyros Papaloukas', 'Nicolae Darascu', 'Albert Marquet', 'Ion Theodorescu-Sion']

In [4]:
all_metadata['selected'] = all_metadata['artist'].apply(lambda x: x in artists)

In [5]:
metadata = all_metadata[all_metadata['selected'] == True]

In [6]:
metadata = metadata.drop('selected', axis=1)

In [7]:
metadata.head()

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
15,Paul Serusier,1890,genre painting,7099.0,5857.0,9803854.0,wikiart,Cloisonnism,Seaweed Gatherer,train_and_test,False,32996.jpg
41,Georges Seurat,1884,NaN,6367.0,4226.0,11579390.0,wikipedia,Pointillism,Bathers at Asnières,train_and_test,True,39751.jpg
65,Paul Signac,NaN,cityscape,5616.0,4312.0,10612858.0,wikiart,Pointillism,View of the Port of Marseilles,train_and_test,True,74221.jpg
69,Georges Seurat,1884,genre painting,5910.0,4001.0,5330653.0,wikiart,Pointillism,Study for A Sunday on La Grande Jatte,train_and_test,True,31337.jpg
96,Gustave Caillebotte,1881,genre painting,5164.0,4087.0,3587461.0,wikiart,Impressionism,Rising Road,train_and_test,False,29616.jpg


In [8]:
metadata.to_csv('data/metadata.csv', index=None)

In [9]:
df = pd.read_csv('data/metadata.csv')

In [10]:
df.head()

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,Paul Serusier,1890,genre painting,7099.0,5857.0,9803854.0,wikiart,Cloisonnism,Seaweed Gatherer,train_and_test,False,32996.jpg
1,Georges Seurat,1884,NaN,6367.0,4226.0,11579390.0,wikipedia,Pointillism,Bathers at Asnières,train_and_test,True,39751.jpg
2,Paul Signac,NaN,cityscape,5616.0,4312.0,10612858.0,wikiart,Pointillism,View of the Port of Marseilles,train_and_test,True,74221.jpg
3,Georges Seurat,1884,genre painting,5910.0,4001.0,5330653.0,wikiart,Pointillism,Study for A Sunday on La Grande Jatte,train_and_test,True,31337.jpg
4,Gustave Caillebotte,1881,genre painting,5164.0,4087.0,3587461.0,wikiart,Impressionism,Rising Road,train_and_test,False,29616.jpg


In [69]:
train_df = df[df['in_train'] == True]
train_df = train_df.reset_index(drop=True)

In [74]:
test_df = df[df['in_train'] == False]
test_df = test_df.reset_index(drop=True)

## Create matched pairs dataframe

In [113]:
# create training pairs df
col_names = ['id1', 'id2', 'filename_1', 'filename_2', 'same_artist']

pairs = []
for i in range(len(train_df)):
    for j in random.choices(range(len(train_df)), k=36):
        row = (i, j, f"data/train_224_crop/{train_df.iloc[i, 11]}", f"data/train_224_crop/{train_df.iloc[j, 11]}", (train_df.iloc[i, 0] == train_df.iloc[j, 0]))
        pairs.append(row)

all_train_pairs_df = pd.DataFrame(pairs, columns=col_names)

In [88]:
# create test pairs df
pairs = []
for i in range(len(test_df)):
    for j in random.choices(range(len(test_df)), k=36):
        row = (i, j, f"data/test_224_crop/{test_df.iloc[i, 11]}", f"data/test_224_crop/{test_df.iloc[j, 11]}", (test_df.iloc[i, 0] == test_df.iloc[j, 0]))
        pairs.append(row)
        
test_pairs_df = pd.DataFrame(pairs, columns=col_names)

In [114]:
train_pairs_df = all_train_pairs_df.sample(frac=0.8)
val_pairs_df = all_train_pairs_df[~all_train_pairs_df.index.isin(train_df.index)]
train_pairs_df.reset_index(drop=True, inplace=True)
val_pairs_df.reset_index(drop=True, inplace=True)

## Data preprocessing

In [11]:
# resize all of the images to 256x256
for idx in tqdm(range(len(df))):
    row = df.iloc[idx]
    fname = row['new_filename']
    if row['in_train'] == True:
        old_path = 'data/my_train'+'/'+fname
        new_path = 'data/train_256_border'+'/'+fname
        resize_img(old_path, new_path, 256)
    else:
        old_path = 'data/my_test'+'/'+fname
        new_path = 'data/test_256_border'+'/'+fname
        resize_img(old_path, new_path, 256)

  0%|          | 0/13894 [00:00<?, ?it/s]

KeyError: 'folder'

In [33]:
# center crop all images to 224x224 area
for idx in tqdm(range(len(df))):
    row = df.iloc[idx]
    fname = row['new_filename']
    if row['in_train'] == True:
        old_path = 'data/train_256/'+fname
        new_path = 'data/train_224_crop/'+fname
        center_crop(old_path, new_path, 224)
    else:
        old_path = 'data/test_256/'+fname
        new_path = 'data/test_224_crop/'+fname
        center_crop(old_path, new_path, 224)

  0%|          | 0/13894 [00:00<?, ?it/s]

## Create dataloader

In [105]:
def load_image(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.transpose(img, (2, 0, 1))
    img = torch.tensor(img / 255.).float()
    normalize = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    img = normalize(img)

In [107]:
class ArtistDataset(Dataset):
    def __init__(self, df):
        self.df = df
        
        self.remap = {False: 0, True: 1}
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        img_1 = load_image(row['filename_1'])
        img_2 = load_image(row['filename_2'])
        
        label = torch.tensor(self.remap[row['same_artist']])
        
        return img, label

In [109]:
train_ds = ArtistDataset(train_pairs_df)
train_dl = DataLoader(train_ds, batch_size=int(len(train_ds)/1000), shuffle=True)

In [115]:
val_ds = ArtistDataset(val_pairs_df)
val_dl = DataLoader(val_ds, batch_size=int(len(val_ds)/1000), shuffle=True)

In [116]:
test_ds = ArtistDataset(test_pairs_df)
test_dl = DataLoader(test_ds, batch_size=int(len(test_ds)/1000), shuffle=True)

In [117]:
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
        self.eps = 1e-9

    def forward(self, hidden_x1, hidden_x2, y):
        # euclidian distance
        diff = hidden_x1 - hidden_x2
        dist_sq = torch.sum(torch.pow(diff, 2), 2)
        dist = torch.sqrt(dist_sq + self.eps)

        mdist = self.margin - dist
        dist = torch.maximum(mdist, torch.zeros(mdist.shape))
        loss = ((y * dist_sq) + ((1 - y) * torch.pow(dist, 2))) / 2
        # average loss over a batch
        avg_loss = torch.sum(loss)/hidden_x1.shape[1]
        return avg_loss

In [118]:
model = models.alexnet(pretrained=True)

In [119]:
for param in model.parameters():
    param.requires_grad = False

In [120]:
model.classifier[6] = nn.Linear(4096, 52)

In [121]:
params_to_update = []

for param in model.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self, model, n_artists=52):
        super(SiameseNetwork, self).__init__()
        self.model = model
        self.linear = nn.Linear(n_artists, 1)
        
    def forward(self, x1, x2):
        y_pred_x1 = self.model(x1)
        y_pred_x2 = self.model(x2)
        x = torch.abs(y_pred_x1 - y_pred_x2)
        x = self.linear(x)
        return y_pred_x1, y_pred_x2, x.squeeze()

In [ ]:
def one_pass_dual(model, dataloader, optimizer, loss_1, loss_2, backwards=True, print_loss=False):
    
    if backwards == True:
        model.train()
    else:
        model.eval()
    
    total_loss = 0.0
    avg_accs = []
    avg_iou = []
    for x, y in tqdm(dataloader):
        
        y_pred = model(x)
        y_pred_coords = y_pred[:, :4]

        y_pred_label = y_pred[:, 4:]
        ce_loss = loss_1(y_pred_label, y['label'])
        l1_loss = loss_2(y_pred_coords, y['boxes'])
        loss = ce_loss + l1_loss
        total_loss += loss.item()
        avg_accs.append(torch.sum(torch.argmax(y_pred_label, dim=1) == y['label']).item()/len(y['label']))
        avg_iou.append((sum([iou(y_pred, y) for y_pred, y in zip(y_pred_coords, y['boxes'])])/len(y['boxes'])).item())
        
        if backwards == True:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    avg_loss = total_loss / len(dataloader)
    avg_acc = sum(avg_accs) / len(dataloader)
    avg_iou = sum(avg_iou) / len(dataloader)
    
    if print_loss == True:
        print(avg_loss)
        print(avg_acc)
    
    return avg_loss, avg_acc, avg_iou